In [7]:
import numpy as np
import imageio
import matplotlib.pyplot as plt
import torch as pt

from models.common import PUPieApp, PUPieAppEndToEnd
from loader.dataset import ImageLoader, image2patches

%matplotlib inline

# Set common variables

In [8]:
# Path to reference and distorted iamges
ref_path = './example_images/ldr/i06.bmp'
dist_path ='./example_images/ldr/i06_15_5.bmp'
dynamic_range = 'ldr'

# Parameters of the display model (Assuming peak and black level of a display on which LDR image is shown).
# Set to 100 and 0.5 if unsure. The parameter is not used for HDR images as these are given in luminance values.
lum_top = 100
lum_bottom = 0.5

# The quality assessment model operates on 64x64 patches sampled on a regular grid. 
# The shift specifies the window shift for sampling the patchs. The smaller the shift the more accurate the model is.
stride = 32
saved_state_model = './net_only.pt'
state = pt.load(saved_state_model, map_location='cpu')
loader = ImageLoader()

def read_convert_pt_image(image_path):
    image = imageio.imread(image_path)
    image = pt.from_numpy(imageio.core.asarray(image))
    image = image.permute(2,0,1)
    return image

# Run PU-PieAPP

The network takes as input:

* reference image of the shape [B, C, H, W]
* distorted image of the shape [B, C, H, W]
* dynamic range: {'hdr','ldr'}
* bottom luminance: typical for SDR display is 0.5
* top luminance: typical for SDR display 100-300
* stride: sliding window for 64x64 patches in the image


In [10]:
# Read images and split them into patches (note: no processing is applied). 
image_ref = read_convert_pt_image(ref_path)
image_dis = read_convert_pt_image(dist_path)
image_ref = image_ref.unsqueeze(0)
image_dis = image_dis.unsqueeze(0)


# Create and load the model
net = PUPieAppEndToEnd(state)
net.eval();

# Run the network with no gradient
with pt.no_grad():
    score = net(image_ref, image_dis, im_type=dynamic_range, lum_bottom=lum_bottom, lum_top=lum_top, stride =stride)
    
print('PU-PieAPP Quality Score: ', score.item())

PU-PieAPP Quality Score:  -0.8524742126464844
